## 1、取出数据并显示行号

表名:products

字段说明

1、p_id  自增ID，没啥特别的

2、p_name 商品名称

3、p_type 商品分类 （为了方便演示，直接用中文了）

4、p_view 点击量

![](img/4.1.png)

```sql
select p_name,p_type,p_view from products ORDER BY p_view desc 
```

![](img/4.2.png)

- 实现思路

  MySQL并没有 oracle、sqlserver那样直接的方法。因此我们只能自己实现

  我们引入会话变量

  基本设置方法：

  set @xxx=xxxx;

  select @xxxx;

  只要会话不结束，这个变量就一直存在

- 对变量进行赋值

  := 这才是mysql对变量真正赋值的方式 

  如

  set @age=10;

  select @age:=10;

  只不过使用 set 语句时 可以写成=

  如set @age=10;

- 结合sql语句使用

  ```sql
  select p_name,p_type,p_view,@rownum:=@rownum+1 from products ORDER BY p_view desc
  ```

  **这时会发生一个问题，如果这个变量 一上来没定义，则就一直是null**

- mysql的 ifnull函数

  IFNULL(expr1,expr2) 

  expr1 不为 NULL，则 IFNULL() 的返回值为 expr1; 否则其返回值为 expr2

  ```sql
  select p_name,p_type,p_view, IFNULL(@rownum:=@rownum+1,@rownum:=1)  from products a    ORDER BY p_view desc 
  ```

  问题又来了，第二次运行 @rownum不是从1开始的

- 解决方案

  ```sql
  select  p_name,p_type,p_view,@rownum:=@rownum+1 
  from products a,(select @rownum:=0) b ORDER BY p_view desc
  ```

## 2、分组后在分组内排序、每个分组中取前N条

表名:products

字段说明

1、p_id  自增ID，没啥特别的

2、p_name 商品名称

3、p_type 商品分类 （为了方便演示，直接用中文了）

4、p_view 点击量

![](img/4.3.png)

- 写出分组的SQL

  ```sql
  select a.p_type,a.p_name,a.p_view  from products a GROUP BY p_type,p_name
  ```


- 排序

  ```sql
  select a.p_type,a.p_name,a.p_view  from products a GROUP BY p_type,p_name
  order by p_view desc
  ```

  ![](img/4.png)

- 进一步

  ```sql
  select a.p_type,a.p_name,a.p_view from products a order by a.p_type desc, a.p_view desc
  ```

- 取出前N条

  ```sql
  select p_type,p_name,p_view,IF(@pre=p_type,@rownum:=@rownum+1,@rownum:=1),@pre:=p_type from (select a.p_type,a.p_name,a.p_view from products a  
  GROUP BY a.p_type,a.p_name order by a.p_type desc, a.p_view desc) a,(select @pre:='',@rownum:=0 ) b
  ```

  ![](img/4.5.png)

  **IF函数**

  **IF(条件,表达式1，表达式2)**

  **如果条件成立 则返回表达式1，否则是2**

## 3、纯SQL也能实现小算法（辅助决策）计算商品评分、及时补货
表名:products

字段说明

1、p_id  自增ID

2、p_name 商品名称

3、p_type 商品分类

4、p_view 点击量

![](img/4.6.png)

表名:products_sales

字段说明

1、p_id  商品ID

2、p_name 商品名称

3、p_sales 商品的销量（卖了多少个）

注意：上一张中并不是所有商品都有销量

![](img/4.7.png)

- 写一个SQL。根据分类显示出商品的名称、点击量和销售量情况。没有销售的置为0

  ```sql
  select p_type,a.p_name,a.p_view, IFNULL(b.p_sales,0) from products a 
  LEFT JOIN products_sales b
  on a.p_id=b.p_id
  GROUP BY  a.p_type,a.p_name order by a.p_type desc, a.p_view desc
  ```

- 根据上面的结果显示每个分类的销量平均值

  这时我们就要分拆SQL来实现了, 先实现 每个分类下的 总销售数和总条数计算

  ```sql
  select p_type,round(sum(sales)/count(*),0) as sales_avg from ( select  p_type, a.p_name,a.p_view, IFNULL(b.p_sales,0) as sales from products a 
  LEFT JOIN products_sales b
  on a.p_id=b.p_id
  
  GROUP BY  a.p_type,a.p_name order by a.p_type desc, a.p_view desc ) c
  GROUP BY p_type
  ```

- 上面两个SQL进行合并

  ```sql
  select a.*,b.sales_avg from (select p_type,a.p_name,a.p_view, IFNULL(b.p_sales,0) as sales from products a 
  LEFT JOIN products_sales b
  on a.p_id=b.p_id
  GROUP BY  a.p_type,a.p_name order by a.p_type desc, a.p_view desc 
  ) a ,
  (select p_type,round(sum(sales)/count(*),0) as sales_avg from (select p_type, a.p_name,a.p_view, IFNULL(b.p_sales,0) as sales from products a  
     LEFT JOIN products_sales b
  on a.p_id=b.p_id
  GROUP BY  a.p_type,a.p_name order by a.p_type desc, a.p_view desc ) c
  GROUP BY p_type
  )
  b
  where a.p_type=b.p_type
  ```

- 再加入商品的分类点击量平均值

  ```sql
  select a.*,b.sales_avg,c.view_avg from (select p_type,a.p_name,a.p_view, IFNULL(b.p_sales,0) as sales from products a  
     LEFT JOIN products_sales b
  on a.p_id=b.p_id
  
  GROUP BY  a.p_type,a.p_name order by a.p_type desc, a.p_view desc 
  ) a ,
  ( select p_type,round(sum(sales)/count(*),0) as sales_avg from ( select  p_type, a.p_name,a.p_view, IFNULL(b.p_sales,0) as sales from products a  
     LEFT JOIN products_sales b
  on a.p_id=b.p_id
  GROUP BY  a.p_type,a.p_name order by a.p_type desc, a.p_view desc ) c
  GROUP BY p_type
  )
  b ,
  (select p_type,round(sum(p_view)/count(*),0) as view_avg from ( select  p_type, a.p_name,a.p_view, IFNULL(b.p_sales,0) as sales from products a  
     LEFT JOIN products_sales b
  on a.p_id=b.p_id
  GROUP BY  a.p_type,a.p_name order by a.p_type desc, a.p_view desc ) c
  GROUP BY p_type) c
  
  where a.p_type=b.p_type and a.p_type=c.p_type
  ```

  ![](img/4.8.png)

- 实现方法

  分别把 计算各自的 

  1、点击量/点击量均值  

  2、销售量/销售量均值

  两者相加，可以得到一个简单评分

  又有问题了，猪肉的评分不应该比五花肉多。

  因此我们要加入简单的权重，譬如点击量评分占30%、70%

  ```sql
  select p_type,p_name, (p_view/view_avg)*0.3+(sales/sales_avg)*0.7 from (
  select a.*,b.sales_avg,c. view_avg  from (select   p_type,a.p_name,a.p_view, IFNULL(b.p_sales,0) as sales from products a  
     LEFT JOIN products_sales b
  on a.p_id=b.p_id
  
  GROUP BY  a.p_type,a.p_name order by a.p_type desc, a.p_view desc 
  ) a ,
  ( select p_type,round(sum(sales)/count(*),0) as sales_avg from ( select  p_type, a.p_name,a.p_view, IFNULL(b.p_sales,0) as sales from products a  
     LEFT JOIN products_sales b
  on a.p_id=b.p_id
  GROUP BY  a.p_type,a.p_name order by a.p_type desc, a.p_view desc ) c
  where c.sales>0
  GROUP BY p_type
  )
  b,
  (select p_type,round(sum(p_view)/count(*),0) as view_avg from products GROUP BY p_type )
  c
  where a.p_type=b.p_type  and a.p_type=c.p_type ) aa
  ```

  ![](img/4.8.png)

  

## 4、mysql中自连接的使用

![](./img/4.a.png)

表名:users

q字段说明

1、uid  自增ID

2、name 用户名

3、reco_id  推荐人的id

- 场景说明

  1、做一个用户激励注册系统

  2、用户注册时可以填 “推荐人”

  ​    于是我们很可能会有个需求:

     查询出，注册用户的推荐人，给予一定奖励

- 这时我们可以使用自连接

  **所谓的自连接。就是自己和自己进行inner join或 left join**

  ```sql
  select  a.u_name as username,b.u_name as refer from webusers a 
  INNER JOIN webusers b 
  on a.p_id=b.id;
  ```

  ![](./img/4.b.png)

- 进一步处理

  ```sql
  select GROUP_CONCAT(name ORDER BY id desc SEPARATOR '|') as u,reco_id from users GROUP BY reco_id;
  ```

  ![](./img/4.c.png)

  其中函数：

  - GROUP_CONCAT

    通常和group by 一起使用，把相同的分组的字段值连接起来

- 最终效果

  ```sql
  select a.users,b.u_name from (
  select GROUP_CONCAT(u_name) as users,p_id from webusers   
  GROUP BY p_id ) a INNER JOIN webusers b
  on a.p_id =b.u_id
  ```

  ![](./img/4.d.png)

## 5、删除重复数据

![](img/4.e.png)

表名:reviews 

字段说明:

1、id  自增ID

2、content  评论内容

3、userid  评论人id

4、news_id 对应的新闻id

- 内容查看

  ```sql
  select r_content,r_userid,count(*) from reviews GROUP BY r_content,r_userid
  ```

  ![](img/4.f.png)

- 筛选出count大于1的值

  ```sql
  select r_content,r_userid,count(*) from reviews GROUP BY r_content,r_userid
  HAVING count(*)>1
  ```

  ![](img/4.g.png)

- 找到重复的记录并删掉:

  通常做一个inner join

  ```sql
  select a.* from reviews a INNER JOIN
    (select r_content,r_userid,count(*) from reviews GROUP BY r_content,r_userid having count(*)>1 ) b
   on a.r_content=b.r_content and a.r_userid=b.r_userid;
  ```

- 加入行号

  ```sql
  select a.*,if(@tmp=CONCAT(r_content,r_userid),@rownum:=@rownum+1,@rownum:=1) as rownum,@tmp:=CONCAT(r_content,r_userid) from (select a.* from reviews a INNER JOIN (select r_content,r_userid,count(*) from reviews GROUP BY r_content,r_userid having count(*)>1) b
   on a.r_content=b.r_content and a.r_userid=b.r_userid) a,(select @rownum:=0,@tmp:='') b;
  ```
  
  ![](img/4.h.png)

- 找出重复的数据并删除

  ```sql
  select r_id from 
  (select a.*,if(@tmp=CONCAT(r_content,r_userid),@rownum:=@rownum+1,@rownum:=1) as rownum,@tmp:=CONCAT(r_content,r_userid) from 
  (select a.* from reviews a INNER JOIN 
  (select r_content,r_userid,count(*) from reviews GROUP BY r_content,r_userid having count(*)>1 ) b
   on a.r_content=b.r_content and a.r_userid=b.r_userid) a,(select @rownum:=0,@tmp:='') b) a where rownum>1;
  ```

  ![](img/4.i.png)

- 利用group_concat

  ```sql
  select GROUP_CONCAT(r_id) as ids,r_content,r_userid,count(*) from reviews GROUP BY r_content,r_userid
  HAVING count(*)>1
  ```

  ![](img/4.j.png)

## 6、有重复数据不插入或更新的处理方法

表名:news

字段说明

1、news_id  自增ID

2、news_title 新闻标题

3、news_abstract 新闻摘要

4、news_code

![](img/4.k.png)

- 场景

    譬如管理员后台

  1、编辑在后台手工添加新闻

  2、或者用爬虫抓到新闻后插入

  由于一些不可描述的原因。我们可能需要会插入相同的新闻

  ```sql
  insert into news(news_id,news_title,news_abstract,news_code)
  VALUES(1,'新闻标题1','新闻摘要1','a');
  ```

- 利用news_code字段。

  在程序拼凑SQL语句时，执行一个md5 过程,

  让news_code值=md5(标题的内容+摘要的内容).我们使用sql来表示

  ```sql
  insert into news(news_title,news_abstract,news_code)
  VALUES('新闻标题1','新闻摘要1'
  ,MD5(CONCAT('新闻标题1','新闻摘要1')));
  ```

  同时我们设置news_code字段为唯一索引

  ![](img/4.l.png)

- 扩展

  如果当新闻被重复插入时，我们需要统计次数。以此来判断我们的管理

  员或爬虫是否“失误的次数太多”

  

  这时我们加入一个字段，叫做dupnum，int型 (用来记录重复的次数)

- ON DUPLICATE KEY UPDATE(mysql特有的语法)

  一般跟在insert 后面出现。 如果insert会导致UNIQUE索引或PRIMARY KEY中出现重复值，则在出现重复值的行执行UPDATE

  ```sql
  insert into news(news_title,news_abstract,news_code)
  VALUES('新闻标题2','新闻摘要2'
  ,MD5(CONCAT('新闻标题2','新闻摘要2')))
  on DUPLICATE key 
  update dupnum=dupnum+1
  ```

  - 常见场景

    表名:users

    字段说明

    1、user_id  自增ID

    2、user_name 用户名，唯一索引

    3、user_qq 用户QQ

    4、update_time  用户信息最后更新时间

    ![](img/4.m.png)

  - 插入记录时

    ```sql
    insert into users(user_name,user_qq) values('ccy','123')
    ```

    一旦有用户更新记录，则

    ```sql
    insert into user_info(user_name,user_qq) values('ccy','345') 
    on DUPLICATE key update update_time=now(),user_qq=values(user_qq);
    ```

## 7、update表子查询、多条件判断

表名:user_level 用户级别

字段说明

1、id  自增ID表名:user_level 用户级别

字段说明

1、id  自增ID

2、user_name 用户名

3、user_total 消费总金额

4、user_rank 用户等级、默认是吃瓜

![](img/4.n.png)

- 场景

  系统运行了一段时间后，好多用户消费了

  我们需要在表中对用户进行等级更新,我们的需求是

  1、只对超过平均消费金额的用户进行等级升级

  2、达到平均消费金额 1倍的用户 等级是 白金用户

  3、2倍或以上的是黄金用户

  4、其他一律是吃瓜用户

- 思路：

  求平均

  select avg(user_total) from user_level

  更新

   update user_level set user_rank= xxxooo  where user_total >= 平均数

- mysql 里面的case when

  case 

    when **表达式** **then****表达式** 

    **else** **表达式**

   **end**

  **往往用于****select** **查询时 对字段进行特殊条件处理**

  ```sql
  select *,
  case user_total 
   when 100 then '消费正好满100的用户'
   else  '其他'
   end 
   from user_level
   
   select *,
  case   
   when user_total> 50 and user_total<100 then '消费超过50的用户'
  when user_total> 100 then '消费超过100的用户'
   else  '其他'
   end 
   from user_level
  ```

- update里面也可以使用

  ```sql
  update user_level,(select avg(user_total) as avg from user_level) b set user_rank=
  case
   when round(user_total/avg)>=1 and  round(user_total/avg)<2 then '白金用户'
   when round(user_total/avg)>=2  then '黄金用户'
  
  ELSE
   '吃瓜'
  end  where user_total>=b.avg
  ```

## 8、利用order by 实现"排名作弊"

表名:user_level 用户级别

字段说明

1、id  自增ID，没啥特别的

2、user_name 用户名

3、user_total 消费总金额

4、user_rank 用户等级

![](img/4.o.png)

- 场景

  有时候我们需要在网站中实现排行榜，譬如用户消费排行、商品销量排

  行、新闻点击排行等。

   排行数据免不了要真假掺半

  结合表我们的需求是

  1、根据字段 user_total倒排序

  2、其中id为2,4,6的用户为我们“内部用户”,要直挺挺的置顶

- 常规做法

  我们首先想到的是union

  ```sql
  select * from (select *  from user_level where id in (2,4,6) order by id desc ) a
  union
  select * from (select *  from user_level where id not in (2,4,6) order by id desc ) b
  ```

- 新做法

  ```sql
  select * from user_level order by  id in(4,6,2) and id<>2 desc,user_total desc
  ```

## 9、获取连续签到X天用户列表

表名:user_sign 用户签到表 

字段说明

1、id  自增ID，没啥特别的

2、user_name 用户名

3、sign_date 签到日期

![](img/4.p.png)

- 场景

  有时候我们在网站中要假模假样的做一些签到功能。让用户可以天天来

  点一次网站，以增加我们网站的活跃率。

  那么其中一个功能是：

   统计出连续签到X天的用户。然后对他们进行排行或者

  假模假样的奖励

- 分解步骤

  - 分组排序

    ```sql
    select user_name,sign_date from user_sign
    GROUP BY user_name,sign_date ORDER BY user_name,sign_date
    ```

  - 加入分组行号

    ```sql
    select user_name,sign_date,IF(@pre=user_name,@rownum:=@rownum+1,@rownum:=1),
    @pre:=user_name 
     from (
    select user_name,sign_date from user_sign
    GROUP BY user_name,sign_date ORDER BY user_name   ,sign_date  ) a ,(select @pre:='',@rownum:=0  ) b 
    ```

  - 计算相邻两行的日期是否 相差一天

    mysql的函数 datediff(date1,date2)

    返回两个日期之间的天数 注意是 date1-date2

    ```sql
    select user_name,sign_date,IF(@pre=user_name and DATEDIFF(sign_date,@pre_date)=1,@rownum:=@rownum+1,@rownum:=1),
     
    @pre:=user_name,@pre_date:=sign_date
     from (
    select user_name,sign_date from user_sign
    GROUP BY user_name,sign_date ORDER BY user_name   ,sign_date  ) a ,(select @pre:='',@rownum:=0,@pre_date:='' ) b
    ```

## 10、子查询去重、获取商品分类最新销售情况

表名:prod_sales 用户签到表 

字段说明

1、id  自增ID，没啥特别的

2、prod_class 商品分类

3、sales_date 销售汇总日期

4、prod_id 商品ID或名称

5、sales_num 销售数量

这类统计在项目后台经常出现。属于统计后二次加工统计

![](img/4.r.png)

其实这是统计出 某个商品在某个日期（天）内的销售总数，所以肯定有重复。

需求是：**查询出 图书和食品两个分类（或多个分类） 在****最新的****一天内的商品销售情况** 

![](img/4.s.png)

- 做法1

  ```sql
  select prod_class,prod_id, max(sales_date) as sn from prod_sales
  GROUP BY prod_class 
  或
  select prod_class,prod_id, max(sales_date) as sn from prod_sales
  GROUP BY prod_class,prod_id
  ```

- 做法2

  - 找出各个分类中  最新有销售的日期

    ```sql
    select prod_class,  max(sales_date) as sn from prod_sales
    GROUP BY prod_class 
    ```

  - 合并

    ```sql
    select a.* from prod_sales a INNER JOIN
    (select prod_class,max(sales_date) as sn from prod_sales
    GROUP BY prod_class) b 
    on a.prod_class=b.prod_class and a.sales_date=b.sn  order by prod_class
    ```

##11、多表关联update（用户积分奖励)

表名:users_score 用户积分存储表 

字段说明

1、id  自增ID

2、user_name 用户名

3、user_score 积分

![](img/4.t.png)

表名:users_buy 用户积分存储表 

字段说明

1、id  自增ID

2、user_name 用户名

3、paymoney 消费金额

4、paydate  消费日期

![](img/4.u.png)

规定在某2天内，凡是消费的客户给予 消费金额的10%作为积分奖励。以最大的一天为准。不累加

![](img/4.v.png)

- 分解

  - 找出消费最大的金额(为了演示方便，省略日期因素)

    ```sql
    select max(paymoney) as mp,user_name from users_buy group by user_name 
    ```

    解法1：

    1、写个存储过程

      利用游标循环，然后一句句update users_score set xxxxx

    2、用程序把上述语句取出来，循环

     继续一句句 update xxxxx

    解法2：

    ```sql
    update users_score a INNER JOIN (select max(paymoney) as mp,user_name from users_buy group by user_name ) b on a.user_name=b.user_name 
    set a.user_score= a.user_score+(b.mp*0.1)
    ```

    